## Procurement of Resources under uncertain demand: a two-stage stochastic programming formulation

In this example, we showcase the code related to Example 14.1 from the book. In the example, we have a set of products $p \in \mathcal{P}$ where $\mathcal{P}=\left\{1,2,3\right\}$. 

To produce one unit of each product, a different combination of resources $r \in \mathcal{R}$ where $\mathcal{R}=\left\{1,2,3\right\}$ is needed. 
We define $Q_{rp}$ the quantity of resource $r$ needed to produce 1 unit of product $p$. We have $Q_{11}=3$, $Q_{12}=1$, $Q_{13}=4$, $Q_{21}=2$, $Q_{22}=2$, $Q_{23}=2$, $Q_{31}=4$, $Q_{32}=2$, and $Q_{33}=0$.

Additionally, every unit of acquired resource costs $C_r$ monetary units.We have $C_1=3$, $C_2=2$, and $C_3=1$. In contrast, every unit of product generates a revenue equivalent to $R_p$. We have $R_1=25$, $R_2=20$, and $R_3=10$.

Unfortunately, the producer does not know exactly the future demand for each product, while decisions on how much of each resource to acquire must be made now. Notwithstanding, they have forecast 3 potential scenarios $s \in \mathcal{S}$ where in each scenario $D_{ps}$ is the demand of product $p$ for the considered scenario $s$. Each scenario is estimated to occur with a probability $P_i$, with $P_1=0.5$, $P_2=0.4$, and $P_3=0.1$. In terms of demand, we have:

- $s=1$: $D_{11}=40$, $D_{21}=30$, $D_{31}=10$ 
- $s=2$: $D_{12}=30$, $D_{22}=20$, $D_{32}=0$
- $s=3$: $D_{13}=10$, $D_{23}=30$, $D_{33}=50$

The producer needs to acquire resources now (**first stage**) and decide how much of each product to produce (**second stage**) when the actual demand is revealed, with the goal of **maximizing profit**. Different strategies are possible to achieve the goal. Here, we will be using 2 strategies:

- **two-stage stochastic programming approach**: this approach maximizes the expected revenue value from the second stage considering each scenario with its defined probability
- **``myopic" deterministic approach**: the producer assumes the expected demand per product is the weighted average according the the 3 scenarios, hence removing the scenarios and dealing with a deterministic model. This model is defined *myopic* because nobody is guaranteeing the producer that the average demand values will be the real ones. For the sake of clarity, the 3 average values are $\overline{D}_1=0.5 \times 40 + 0.4 \times 30 + 0.1 \times 10 = 33$, $\overline{D}_2=0.5 \times 30 + 0.4 \times 20 + 0.1 \times 30 = 26$, and $\overline{D}_3=0.5 \times 10 + 0.4 \times 0 + 0.1 \times 50 = 10$

We will be solving **5 instances**, namely:

1. two-stage stochastic model
2. ``myopic" deterministic model
3. deterministic model where the quantity of resources is fixed and given by instance 2., and the actual demand realization is the one associated with scenario $s=1$
4. deterministic model where the quantity of resources is fixed and given by instance 2., and the actual demand realization is the one associated with scenario $s=2$
5. deterministic model where the quantity of resources is fixed and given by instance 2., and the actual demand realization is the one associated with scenario $s=3$

In instance 2., the producer is still deciding the optimal quantity of each resources to acquire in light of an expected demand that is the weighted average across the 3 scenarios. In instances 3., 4., and 5., we assume that the choice has already been made in instance 2., and the producer's only flexibility left is to produce the 3 products in the best way possible (profit-wise) given the fixed supply of resources.


### Two-stage stochastic model

In this model, we use all the sets and parameters defined above. In addition, we define thw 2 sets of decision variables:

- $x_r \ \forall r \in \mathcal{R}$ (continuous): units of resource $r$ acquired
- $y_{ps} \ \forall p \in \mathcal{P}, s \in \mathcal{S}$ (continuous): units of product $p$ produced in scenario $s$

Note that we assume that decision variable type are continuous (readers might argue that integer-valued decision variables might be more proper, but we stick with continuous in this example). In addition, we assume that every unit of every product that is produced is also sold and hence contributes positively to the profit. in the stochastic setting, we define the mathematical model as:

$\begin{align}
\max & -\sum_{r \in \mathcal{R}}C_r x_r + \sum_{s \in \mathcal{S}}P_s\sum_{p \in \mathcal{P}} R_p y_{ps}  && \label{eq:ex_tssp_1_obj}
\end{align}$

s.t.:

$
\begin{align}
& \sum_{p \in \mathcal{P}} Q_{rp}y_{ps} \leq x_r&& \forall r \in \mathcal{R}, s \in \mathcal{S}  \label{eq:ex_tssp_2_C1}\\
& x_r \in \mathbb{R}_0 && \forall r \in \mathcal{R} \label{eq:ex_tssp_2_C2}\\
& y_{ps} \leq D_{ps} && \forall p \in \mathcal{P}, s \in \mathcal{S} \label{eq:ex_tssp_2_C3}
\end{align}$

where the objective aims a maximizing the expected profit considering the costs incurred into the first stage and the expected revenue from the second stage. The first constraint ensures that in every scenario every product is produced within the supply limit provided by decision variable $x_r$ for every resource $r \in \mathcal{R}$ acquired in the first stage, the second defines the continuous nature of the $x_r$, and the third one limits the number of units per product to be produced in each scenario to the associated $D_{ps}$ value. We proceed now to formulate and solve the model.

We start by importing the needed packages

In [2]:
import numpy as np
import pandas as pd
import os
import networkx as nx
import random
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
from pyomo.environ import *

We define the 3 sets $\mathcal{P}$, $\mathcal{R}$, and $\mathcal{S}$ as dictionaries with all the parameters introduced above to be used later in the definition of the model in pyomo 

In [3]:
########################
### Stochastic model ###
########################

# Set of products
P = {1:{1:3,2:2,3:4,'revenue':25},
     2:{1:1,2:2,3:2,'revenue':20},
     3:{1:4,2:2,3:0,'revenue':10}}

# Set of resources
R = {1:{1:3,2:1,3:4,'cost':3},
     2:{1:2,2:2,3:2,'cost':2},
     3:{1:4,2:2,3:0,'cost':1}}

# Set of scenarios
S = {1:{1:40,2:30,3:10,'prob':0.5},
     2:{1:30,2:20,3:0,'prob':0.4},
     3:{1:10,2:30,3:50,'prob':0.1}}

We initialize the pyomo model and define the 3 sets

In [4]:
# Define optimization model
model = ConcreteModel()

# Define sets
model.Products     = Set(initialize=P.keys())
model.Resources    = Set(initialize=R.keys())
model.Scenarios    = Set(initialize=S.keys())

We now define all the parameters

In [5]:
# Define parameters
model.Q_rp = Param(model.Resources,model.Products, initialize=
             {(r,p):v[p] for r,v in R.items() for p in P.keys()}, within=Any)
model.R_p  = Param(model.Products, initialize={p:v['revenue'] for p,v in P.items()})
model.C_r  = Param(model.Resources, initialize={r:v['cost'] for r,v in R.items()})
model.D_ps = Param(model.Products,model.Scenarios, initialize=
             {(p,s):v[p] for p in P.keys() for s,v in S.items()}, within=Any)
model.P_s  = Param(model.Scenarios, initialize={s:v['prob'] for s,v in S.items()})   


and the 2 sets of decision variables

In [6]:
# Define decision variables
model.x = Var(model.Resources, within=NonNegativeReals)
model.y = Var(model.Products,model.Scenarios, within=NonNegativeReals)

We define the objective as stated above

In [7]:
# Define objective function
model.obj = Objective(expr=-sum(model.C_r[r]*model.x[r] for r in model.Resources)
                      +sum(model.P_s[s]*(sum(model.R_p[p]*model.y[p,s] for p in model.Products)) 
                           for s in model.Scenarios), sense=maximize)

We define the two sets of constraints. We do not need to explicitly enforce $x_r \in \mathbb{R}_0 \ \forall r \in \mathcal{R}$ as it is already accounted for in the definition of the decision variables

In [8]:
# Define constraints
model.max_products_per_resource_scenario = ConstraintList()
for s in model.Scenarios:
    for r in model.Resources:
        model.max_products_per_resource_scenario.add(
                sum(model.Q_rp[r,p]*model.y[p,s] for p in model.Products)<=model.x[r])
        
model.max_products_demand_scenario = ConstraintList()
for s in model.Scenarios:
    for p in model.Products:
        model.max_products_demand_scenario.add(model.y[p,s]<=model.D_ps[p,s])

Finally, we solve the model (here we use Gurobi as te solver. For the open-source variants, just modify the SolverFactory)

In [9]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model) 

{'Problem': [{'Name': 'x13', 'Lower bound': 467.5, 'Upper bound': 467.5, 'Number of objectives': 1, 'Number of constraints': 19, 'Number of variables': 13, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 13, 'Number of nonzeros': 43, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.00515937805176', 'Error rc': 0, 'Time': 2.6482369899749756}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

and output the main results

In [10]:
# Print the results
print('Expected profit with stochastic solution:', model.obj())
print('')

for r in model.Resources:
    print(f'Resource {r}: {model.x[(r)].value}') 
print('')
    
for s in model.Scenarios:
    print('Scenario %i:'%(s))
    for p in model.Products:
        print(f'Product {p}: {model.y[(p,s)].value}') 
    print('')

Expected profit with stochastic solution: 467.5000000007501

Resource 1: 110.0
Resource 2: 113.333333333
Resource 3: 166.666666667

Scenario 1:
Product 1: 26.6666666667
Product 2: 30.0
Product 3: 0.0

Scenario 2:
Product 1: 30.0
Product 2: 20.0
Product 3: 0.0

Scenario 3:
Product 1: 10.0
Product 2: 30.0
Product 3: 12.5



We now solve the myopic deterministic model, which can be written as:

$\begin{align}
\max & -\sum_{r \in \mathcal{R}}C_r x_r +\sum_{p \in \mathcal{P}} R_p y_{p}  && \label{eq:ex_tssp_1_obj}
\end{align}$

s.t.:

$
\begin{align}
& \sum_{p \in \mathcal{P}} Q_{rp}y_{p} \leq x_r&& \forall r \in \mathcal{R} \label{eq:ex_tssp_2_C1}\\
& x_r \in \mathbb{R}_0 && \forall r \in \mathcal{R} \label{eq:ex_tssp_2_C2}\\
& y_{p} \leq \overline{D}_{p} && \forall p \in \mathcal{P} \label{eq:ex_tssp_2_C3}
\end{align}$


Note that now the set of scenarios $\mathcal{S}$ has disappeared. As a consequence, now decision variables $y$ only depend on index $p$, with $y_{p} \ \forall p \in \mathcal{P}$ defining the units of product $p$ produced and sold. The other difference is in the last constraint, where now the right-hand side if the weighted average demand of product $p$ considering the 3 scenarios from the stochastic case 

We directly report the full code, which follows the same logic as before

In [11]:
###########################
### Deterministic model ###    
###########################

# Average demand per product

# Define optimization model
model_det = ConcreteModel()

# Define sets
model_det.Products     = Set(initialize=P.keys())
model_det.Resources    = Set(initialize=R.keys())

# Define parameters
model_det.Q_rp = Param(model_det.Resources,model_det.Products, initialize=
             {(r,p):v[p] for r,v in R.items() for p in P.keys()}, within=Any)
model_det.R_p  = Param(model_det.Products, initialize={p:v['revenue'] for p,v in P.items()})
model_det.C_r  = Param(model_det.Resources, initialize={r:v['cost'] for r,v in R.items()})
model_det.D_p  = Param(model_det.Products, initialize=
             {p:sum(v['prob']*v[p] for _,v in S.items()) for p in P.keys()}, within=Any)

# Define decision variables
model_det.x = Var(model_det.Resources, within=NonNegativeReals)
model_det.y = Var(model_det.Products, within=NonNegativeReals)

# Define objective function
model_det.obj = Objective(expr=-sum(model_det.C_r[r]*model_det.x[r] for r in model_det.Resources)
                      +(sum(model_det.R_p[p]*model_det.y[p] for p in model_det.Products)) 
                           , sense=maximize)

# Define constraints
model_det.max_products_per_resource = ConstraintList()
for r in model_det.Resources:
    model_det.max_products_per_resource.add(
            sum(model_det.Q_rp[r,p]*model_det.y[p] for p in model_det.Products)<=model_det.x[r])
        
model_det.max_products_demand = ConstraintList()
for p in model_det.Products:
    model_det.max_products_demand.add(model_det.y[p]<=model_det.D_p[p])

# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model_det)  

# Print the results
print('Expected profit with deterministic solution:', model_det.obj())
print('')
for r in model.Resources:
    print(f'Resource {r}: {model_det.x[(r)].value}')

Expected profit with deterministic solution: 550.0

Resource 1: 125.0
Resource 2: 118.0
Resource 3: 184.0


Finally, we store the optimal values of the 3 $x_r$ variables to be used in the following models

In [12]:
# Storing the computed resource quantities from the deterministic model 
# to be used later in the three subsequent models where each scenario is
# the one occurring in the "second stage"

Q_r = {r:model_det.x[(r)].value for r in R.keys()} 

We now solve instance 3, where the realized demand is the one from scenario $s=1$. We code the model as follows:

In [13]:
############################################################
### Deterministic model if demand is equal to scenario 1 ###
############################################################

# Define optimization model
model_det_s1 = ConcreteModel()

# Define sets
model_det_s1.Products     = Set(initialize=P.keys())
model_det_s1.Resources    = Set(initialize=R.keys())

# Define parameters
model_det_s1.Q_rp = Param(model_det_s1.Resources,model_det_s1.Products, initialize=
             {(r,p):v[p] for r,v in R.items() for p in P.keys()}, within=Any)
model_det_s1.R_p  = Param(model_det_s1.Products, initialize={p:v['revenue'] for p,v in P.items()})
model_det_s1.C_r  = Param(model_det_s1.Resources, initialize={r:v['cost'] for r,v in R.items()})
model_det_s1.D_p  = Param(model_det_s1.Products, initialize=
             {p:S[1][p] for p in P.keys()}, within=Any)

# Define decision variables
model_det_s1.x = Var(model_det_s1.Resources, within=NonNegativeReals)
model_det_s1.y = Var(model_det_s1.Products, within=NonNegativeReals)

# Define objective function
model_det_s1.obj = Objective(expr=-sum(model_det_s1.C_r[r]*model_det_s1.x[r] for r in model_det_s1.Resources)
                      +(sum(model_det_s1.R_p[p]*model_det_s1.y[p] for p in model_det_s1.Products)) 
                           , sense=maximize)

# Define constraints
model_det_s1.max_products_per_resource = ConstraintList()
for r in model_det_s1.Resources:
    model_det_s1.max_products_per_resource.add(
            sum(model_det_s1.Q_rp[r,p]*model_det_s1.y[p] for p in model_det_s1.Products)<=model_det_s1.x[r])
        
model_det_s1.max_products_demand = ConstraintList()
for p in model_det_s1.Products:
    model_det_s1.max_products_demand.add(model_det_s1.y[p]<=model_det_s1.D_p[p])
    
# Now we also force x decision variables to take the values we computed
# using the deterministic model (hence, they become parameters)
model_det_s1.force_x = ConstraintList()
for r in model_det_s1.Resources:
    model_det_s1.force_x.add(model_det_s1.x[r]==Q_r[r])

Now in the definition of parameters model_det_s1.D_p we use the realized demand from scenario $s=1$. In addition, we added constraint set model_det_s1.force_x to assign the value computed from instance 2. to each $x_r$ (hence, they act as parameters in the following instances).

We now solve instance 3 and plot the main results

In [14]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model_det_s1)  

# Print the results
print('Expected profit with deterministic solution and realized scenario 1:', model_det_s1.obj())
print('')
for p in model.Products:
    print(f'Product {p}: {model_det_s1.y[(p)].value}')
print('')

Expected profit with deterministic solution and realized scenario 1: 550.0

Product 1: 33.0
Product 2: 26.0
Product 3: 0.0



We now solve instance 4, where the only difference with respect to instance 3 is that the realized demand is from scenario $s=2$

In [15]:
############################################################
### Deterministic model if demand is equal to scenario 2 ###
############################################################

# Define optimization model
model_det_s2 = ConcreteModel()

# Define sets
model_det_s2.Products     = Set(initialize=P.keys())
model_det_s2.Resources    = Set(initialize=R.keys())

# Define parameters
model_det_s2.Q_rp = Param(model_det_s2.Resources,model_det_s2.Products, initialize=
             {(r,p):v[p] for r,v in R.items() for p in P.keys()}, within=Any)
model_det_s2.R_p  = Param(model_det_s2.Products, initialize={p:v['revenue'] for p,v in P.items()})
model_det_s2.C_r  = Param(model_det_s2.Resources, initialize={r:v['cost'] for r,v in R.items()})
model_det_s2.D_p  = Param(model_det_s2.Products, initialize=
             {p:S[2][p] for p in P.keys()}, within=Any)

# Define decision variables
model_det_s2.x = Var(model_det_s2.Resources, within=NonNegativeReals)
model_det_s2.y = Var(model_det_s2.Products, within=NonNegativeReals)

# Define objective function
model_det_s2.obj = Objective(expr=-sum(model_det_s2.C_r[r]*model_det_s2.x[r] for r in model_det_s2.Resources)
                      +(sum(model_det_s2.R_p[p]*model_det_s2.y[p] for p in model_det_s2.Products)) 
                           , sense=maximize)

# Define constraints
model_det_s2.max_products_per_resource = ConstraintList()
for r in model_det_s2.Resources:
    model_det_s2.max_products_per_resource.add(
            sum(model_det_s2.Q_rp[r,p]*model_det_s2.y[p] for p in model_det_s2.Products)<=model_det_s2.x[r])
        
model_det_s2.max_products_demand = ConstraintList()
for p in model_det_s2.Products:
    model_det_s2.max_products_demand.add(model_det_s2.y[p]<=model_det_s2.D_p[p])
    
# Now we also force x decision variables to take the values we computed
# using the deterministic model (hence, they become parameters)
model_det_s2.force_x = ConstraintList()
for r in model_det_s2.Resources:
    model_det_s2.force_x.add(model_det_s2.x[r]==Q_r[r])

# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model_det_s2)  

# Print the results
print('Expected profit with deterministic solution and realized scenario 2:', model_det_s2.obj())
print('')
for p in model.Products:
    print(f'Product {p}: {model_det_s2.y[(p)].value}')
print('')

Expected profit with deterministic solution and realized scenario 2: 355.0

Product 1: 30.0
Product 2: 20.0
Product 3: 0.0



We finish with instance 5, where the realized demand is from scenario $s=3$

In [16]:
############################################################
### Deterministic model if demand is equal to scenario 3 ###
############################################################

# Define optimization model
model_det_s3 = ConcreteModel()

# Define sets
model_det_s3.Products     = Set(initialize=P.keys())
model_det_s3.Resources    = Set(initialize=R.keys())

# Define parameters
model_det_s3.Q_rp = Param(model_det_s3.Resources,model_det_s3.Products, initialize=
             {(r,p):v[p] for r,v in R.items() for p in P.keys()}, within=Any)
model_det_s3.R_p  = Param(model_det_s3.Products, initialize={p:v['revenue'] for p,v in P.items()})
model_det_s3.C_r  = Param(model_det_s3.Resources, initialize={r:v['cost'] for r,v in R.items()})
model_det_s3.D_p  = Param(model_det_s3.Products, initialize=
             {p:S[3][p] for p in P.keys()}, within=Any)

# Define decision variables
model_det_s3.x = Var(model_det_s3.Resources, within=NonNegativeReals)
model_det_s3.y = Var(model_det_s3.Products, within=NonNegativeReals)

# Define objective function
model_det_s3.obj = Objective(expr=-sum(model_det_s3.C_r[r]*model_det_s3.x[r] for r in model_det_s3.Resources)
                      +(sum(model_det_s3.R_p[p]*model_det_s3.y[p] for p in model_det_s3.Products)) 
                           , sense=maximize)

# Define constraints
model_det_s3.max_products_per_resource = ConstraintList()
for r in model_det_s3.Resources:
    model_det_s3.max_products_per_resource.add(
            sum(model_det_s3.Q_rp[r,p]*model_det_s3.y[p] for p in model_det_s3.Products)<=model_det_s3.x[r])
        
model_det_s3.max_products_demand = ConstraintList()
for p in model_det_s3.Products:
    model_det_s3.max_products_demand.add(model_det_s3.y[p]<=model_det_s3.D_p[p])
    
# Now we also force x decision variables to take the values we computed
# using the deterministic model (hence, they become parameters)
model_det_s3.force_x = ConstraintList()
for r in model_det_s3.Resources:
    model_det_s3.force_x.add(model_det_s3.x[r]==Q_r[r])

# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model_det_s3)  

# Print the results
print('Expected profit with deterministic solution and realized scenario 3:', model_det_s3.obj())
print('')
for p in model.Products:
    print(f'Product {p}: {model_det_s3.y[(p)].value}')
print('')

Expected profit with deterministic solution and realized scenario 3: 217.5

Product 1: 10.0
Product 2: 30.0
Product 3: 16.25



We now provide two important Key Performance Indicators (KPIs), namely the Expected Value of Perfect Information (EVPI) and the Value of the Stochastic Solution (VSS). The EVPI defines the increase (for a maximization problem) in the objective if we were guaranteed the revealed demand is the weighted average of the 3 scenarios. We compute the EVPI as the difference between the objective value from instance 2 and instance 1. The value is computed below:

In [17]:
EVPI = model_det.obj()-model.obj()
print('')
print('The EVPI value is %i'%(EVPI))
print('')


The EVPI value is 82



Conversely, the VSS is computed as the difference between the objective value of instance 1 (where the model accounts for stochasticity) and the average of the objectives of instances 3, 4, and 5. In fact, in instances 3,4, and 5 we use a level of resources that was computed assuming the average demand is the expected one, while in each instance the demand from scenarios 1, 2, and 3 (respectively) appears. Hence, we solve three deterministic problems and compute the weighted average. We compute the VSS as shown below:

In [21]:
VSS =  model.obj()-(S[1]['prob']*model_det_s1.obj()+S[2]['prob']*model_det_s2.obj()+S[3]['prob']*model_det_s3.obj())
print('')
print('The VSS value is %i'%(VSS))
print('')


The VSS value is 28

